# Import pkgs

In [1]:
import tensorflow as tf
from tensorflow import keras
print("TF version: {}.".format(tf.__version__))
print("Keras version: {}.".format(keras.__version__))

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import cv2
import numpy as np

import segmentation_models as sm

TF version: 2.2.0.
Keras version: 2.3.0-tf.
Segmentation Models: using `tf.keras` framework.


# 參數設定
## [Model ref](https://github.com/qubvel/segmentation_models)

In [2]:
import json
jsonFile = r'F:\SurgeryAnalytics\AI_Cases\Dentistry\1_Linknet_efficientnetb7_0_3\B_AI_training\Dentistry_config.json'

with open(jsonFile,"r") as f:
    config = json.load(f)
config

{'surgeryName': 'Dentistry',
 'label_folder_path': 'F:\\SurgeryAnalytics\\AI_Cases\\Dentistry\\1_Linknet_efficientnetb7_0_3\\0_Data\\0_ori',
 'trans_folder_path': 'F:\\SurgeryAnalytics\\AI_Cases\\Dentistry\\1_Linknet_efficientnetb7_0_3\\0_Data\\1_trans',
 'split_folder_path': 'F:\\SurgeryAnalytics\\AI_Cases\\Dentistry\\1_Linknet_efficientnetb7_0_3\\0_Data\\2_split',
 'correct_label': ['Alveolar_bone',
  'Caries',
  'Crown',
  'Dentin',
  'Enamel',
  'Implant',
  'Mandibular_alveolar_nerve',
  'Maxillary_sinus',
  'Periapical_lesion',
  'Post_and_core',
  'Pulp',
  'Restoration',
  'Root_canal_filling'],
 'detect_label_list': ['Alveolar_bone',
  'Caries',
  'Crown',
  'Dentin',
  'Enamel',
  'Implant',
  'Mandibular_alveolar_nerve',
  'Maxillary_sinus',
  'Periapical_lesion',
  'Post_and_core',
  'Pulp',
  'Restoration',
  'Root_canal_filling'],
 'model': 'Linknet',
 'BACKBONE': 'efficientnetb7',
 'num_classes': 15,
 'save_ckpt_Folder': 'f:\\SurgeryAnalytics\\AI_Cases\\Dentistry\\1_Link

In [3]:
split_folder_path=config["split_folder_path"]+'/test/img'
img_folder_path=split_folder_path
# img_folder_path = r'test/img' # 要預測的圖片資料夾路徑
# save_pred_img_folder_path = r'prediction' # 儲存預測圖片的根目錄資料夾路徑
save_pred_img_folder_path =config["save_pred_img_folder_path"]# 儲存預測圖片的根目錄資料夾路徑
model_name = 'Linknet' # 使用的模型名稱 (Unet/FPN/Linknet/PSPNet)
# backbone_name = 'efficientnetb5' # 使用的backbone名稱
backbone_name=config["BACKBONE"]# 使用的backbone名稱

num_classes = 11 # 模型預測的類別數量, 此數值要+1(for unlabel)
num_classes = config["num_classes"]
correct_label=config["correct_label"]
# correct_label  =['Adenoma',
#                  'Ascites',
#                  'Colon',
#                  'Myoma',
#                  'Other_tissue',
#                  'Pubic',
#                  'Rectum',
#                  'Spine',
#                  'Urinary_bladder',
#                  'Uterus']
color_list=config["color_list"]
# color_list = [
#               [35,119,105],#'Adenoma',
#               [215,148,0],#'Ascites',
#               [12,160,244],#'Colon',
#               [0,208,178],#'Myoma',
#               [148,57,0],#'Other_tissue',
#               [126,187,100],#Pubic
#               [4,84,234],#'Rectum',
#               [71,29,146],#Spine
#               [75,75,152],#'Urinary_bladder',
#               [158,95,229]#'Uterus'
#              ]
#               # 針對資料前處理的correct_label清單, 依序填入想要顯示的BGR色碼 (註:是BGR不是RGB), background顏色不用填.

ckpt_path=config["save_ckpt_Folder"]+"/Linknet_n33_0.81_model.h5"



# ckpt_path = r'model/Linknet_69_0.37_model.h5' # 存放model weight的路徑

# 定義預測函數

In [4]:
def pred_img(img_path, save_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hight, width, _ = img.shape
    resize_img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC) # 註: 若模型輸出寬高非640x480, 則在此修改.
    resize_img = preprocess_input(resize_img)
    resize_img = np.expand_dims(resize_img, axis=0)
    pred_logit = model.predict(resize_img) # shape: (1, 480, 640, 3)
    pred_label = tf.argmax(pred_logit, axis=-1).numpy() # shape: (1, 480, 640)
    pred_label = np.squeeze(pred_label, axis=0) # shape: (480, 640)
    pred_img = np.zeros([512, 512, 3]) # 註: 若模型輸出寬高非640x480, 則在此修改.
    
    for i in np.arange(1, len(color_list)+1):
        idx = np.where(pred_label == i)
        pred_img[idx] = color_list[i-1]
    pred_img = cv2.resize(pred_img, (width, hight), interpolation=cv2.INTER_NEAREST) 
    cv2.imwrite(save_path, pred_img)

# Main

In [6]:
activation = 'sigmoid' if num_classes == 1 else 'softmax'
preprocess_input = sm.get_preprocessing(backbone_name)

exec('model = sm.' + model_name + '(backbone_name, classes=num_classes, activation=activation)')
model.load_weights(ckpt_path)
print(save_pred_img_folder_path)
# for file in os.listdir(img_folder_path):
#     file_name = os.path.splitext(file)[0]
#     img_path = os.path.join(img_folder_path, file)
#     save_path = os.path.join(save_pred_img_folder_path, file_name + "_pred.png")
#     pred_img(img_path, save_path)
#     print("Predict {} image ssuccessfully!".format(file_name))

f:\SurgeryAnalytics\AI_Cases\Dentistry\1_Linknet_efficientnetb7_0_3\C_pred/prediction/20220719104731
